In [65]:
#!pip install rtmlib==0.0.13
#!pip install opencv-python==4.11.0.86
#!pip install librosa==0.11.0
#!pip install Cython==3.0.12
# Microsoft Visual C++ 14.0 or greater is required.
# Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
# The pip version of madmon is outdated and the github repo has many changes recently
#!pip uninstall -y madmom
#!conda install git+https://github.com/CPJKU/madmom.git
#!cd madmom && python setup.py develop --user
#!pip install pyaudio==0.2.14
#!pip install beatnet==1.1.1
#!pip install soundfile==0.13.1
# Install FFmpeg in Windows
# Source: https://phoenixnap.com/kb/ffmpeg-windows
#!pip cache purge
#!pip install moviepy==2.1.1
#!pip install onnxruntime==1.21.1

In [1]:
import os

move_filename = 'e3cbffd'
moves_filepath = os.path.join('outputs', 'moves.json')

In [3]:
!ffmpeg -i inputs/videos/e3cbffd.mp4 -ab 160k -ac 2 -ar 44100 -vn processed/videos/e3cbffd.wav

ffmpeg version 2025-04-23-git-25b0a8e295-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev3, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx

In [5]:
from BeatNet.BeatNet import BeatNet
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
audio_filepath = os.path.join('processed', 'videos', f'{move_filename}.wav')
estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)
beats = estimator.process(audio_filepath)
beats

C:\Users\user\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\user\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\user\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


array([[0.76, 1.  ],
       [1.72, 2.  ],
       [2.68, 1.  ],
       [3.66, 2.  ],
       [4.66, 1.  ],
       [5.6 , 2.  ],
       [6.58, 1.  ],
       [7.58, 2.  ],
       [8.58, 1.  ]])

In [17]:
# 9da1eb8 = 4
# 5db13cd = 1
# 5d80c9a = 1
# 022dccf = - 1

In [7]:
offset = -1
# Fix the beat numbers
for index in range(len(beats)):
    beats[index][1] = int((index + offset) % 8)
beats

array([[0.76, 7.  ],
       [1.72, 0.  ],
       [2.68, 1.  ],
       [3.66, 2.  ],
       [4.66, 3.  ],
       [5.6 , 4.  ],
       [6.58, 5.  ],
       [7.58, 6.  ],
       [8.58, 7.  ]])

In [59]:
''''''
offset = 0
beat_start = 0.76
beat_end = 8.58
num_beats = 17
beat_delta = (beat_end - beat_start) / (num_beats - 1)
beats = [[round(beat_start + x * beat_delta, 2), (x + offset) % 8] for x in range(num_beats)]
beats

[[0.76, 0],
 [1.25, 1],
 [1.74, 2],
 [2.23, 3],
 [2.71, 4],
 [3.2, 5],
 [3.69, 6],
 [4.18, 7],
 [4.67, 0],
 [5.16, 1],
 [5.65, 2],
 [6.14, 3],
 [6.62, 4],
 [7.11, 5],
 [7.6, 6],
 [8.09, 7],
 [8.58, 0]]

In [11]:
'''
# Fix the beat numbers
for index in range(12, 16):
    beats[index][1] = beats[index][1] - 4
beats
'''

'\n# Fix the beat numbers\nfor index in range(12, 16):\n    beats[index][1] = beats[index][1] - 4\nbeats\n'

In [61]:
from moviepy import VideoFileClip, TextClip, CompositeVideoClip

video_file = f"inputs/videos/{move_filename}.mp4"
output_file = f"processed/videos/{move_filename}_counts.mp4"


# Load the video
video_clip = VideoFileClip(video_file)

start_counting = True
text_clips = [video_clip]

for index, x in enumerate(beats):
    start_time = x[0]
    end_time = beats[index + 1][0] if index + 1 < len(beats) else x[0]

    start_counting = start_counting or x[1] == 1

    if start_counting:
        # Create the text clip
        text_clip = TextClip(font='arial.ttf', text=f'{int(x[1] + 1)}', font_size=70, color='white', duration=end_time - start_time)
        
        # Set the duration and position of the text clip
        text_clip = text_clip.with_position(('center', 'bottom'))
        
        # Set the start time of the text clip
        text_clip = text_clip.with_start(start_time)

        text_clips.append(text_clip)
        
# Composite the text clip on the video
video_clip = CompositeVideoClip(text_clips)

# Write the output video file
video_clip.write_videofile(output_file, fps=30, codec='libx264')

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf62.3.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 4442, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc62.11.100 libx264'}}], 'input_number': 0}], 'duration': 8.76, 'bitrate': 4578, 'start': 0.0, 'default_audio_input_number': 0, 'default_audio_stream_number': 0, 'audio_fps': 44100, 'audio_bitrate': 128, 'default_video_input_number': 0, 'default_video_stream_number': 1, 'video_codec_na

MoviePy - Done.
MoviePy - Writing video processed/videos/e3cbffd_counts.mp4



MoviePy - Done !
MoviePy - video ready processed/videos/e3cbffd_counts.mp4


In [63]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# After verifying that the counts are Okay, I can save the counts
data[move_filename] = []
section = 1
for index, x in enumerate(beats):
    count = int(x[1] + 1)
    print(count)
    data[move_filename].append({
        'section': section, 
        'count': count,
        'time': x[0],
        'leader_keypoints': [],
        'follower_keypoints': [],
        'offset': offset
    })
    if count == 8:
        section += 1

with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
1


In [65]:
import cv2

start_counting = False
section = 1
for index, x in enumerate(beats):
    count = int(x[1] + 1)
    print(count)
    vidcap = cv2.VideoCapture(video_file)
    vidcap.set(cv2.CAP_PROP_POS_MSEC, int(x[0] * 1000))      # just cue to position
    success, image = vidcap.read()
    if success:
        cv2.imwrite(os.path.join('processed', 'images', f"{move_filename}_section_{section}_count_{count}.jpg"), image)     # save frame as JPEG file

    # Release resources
    vidcap.release()
    cv2.destroyAllWindows()

    if count == 8 or (count == 4 and index < len(beats) - 2 and beats[index + 1][1] == 0):
        section += 1

1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
1


In [69]:
import numpy as np
import cv2
import os
from rtmlib import Body, draw_skeleton
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

device = 'cpu'
backend = 'onnxruntime'  # opencv, onnxruntime, openvino

cap = cv2.VideoCapture(0)

openpose_skeleton = False  # True for openpose-style, False for mmpose-style

body = Body(
    pose='rtmo',
    to_openpose=openpose_skeleton,
    mode='performance',  # balanced, performance, lightweight
    backend=backend,
    device=device
)
# TODO: Nowadays, I do this by hand. We need to automate it.
leader_detected_first = [
    False,
    True,
    False,
    True,
    True,
    False,
    True,
    False,
    True,
    True,
    False,
    False,
    False,
    False,
    True,
    True,
    True
]
#leader_detected_first = [False for index in range(len(beats))]
img_index = 0

for filename in os.listdir(os.path.join('processed', 'images')):
    # check if the image ends with png
    if (filename.startswith(move_filename)):
        frame = cv2.imread(os.path.join('processed', 'images', filename), cv2.IMREAD_COLOR)
        keypoints, scores = body(frame)

        if leader_detected_first[img_index]:
            data[move_filename][img_index]["leader_keypoints"] = keypoints[0].tolist()
            if keypoints.shape[0] > 1:
                data[move_filename][img_index]["follower_keypoints"] = keypoints[1].tolist()
        elif np.sum(keypoints) > 0:
            data[move_filename][img_index]["follower_keypoints"] = keypoints[0].tolist()
            if keypoints.shape[0] > 1:
                data[move_filename][img_index]["leader_keypoints"] = keypoints[1].tolist()
        print(filename)
        print(keypoints.shape)
        print(np.sum(keypoints))
        print(keypoints[0])

        img_show = frame.copy()
        img_show = draw_skeleton(
            img_show,
            keypoints,
            #np.array([keypoints[0]]),
            scores,
            openpose_skeleton=openpose_skeleton,
            kpt_thr=0.3,
            line_width=2
        )

        cv2.imwrite(os.path.join('processed', 'keypoints', filename), img_show)
        img_index += 1

# Override the data with the keypoints
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

load C:\Users\user\.cache\rtmlib\hub\checkpoints\rtmo-l_16xb16-600e_body7-640x640-b37118ce_20231211.onnx with onnxruntime backend
e3cbffd_section_1_count_1.jpg
(2, 17, 2)
34778.965
[[522.77515 311.03668]
 [521.69836 305.14117]
 [523.4432  305.47797]
 [517.4985  309.60687]
 [548.3888  309.24112]
 [511.9474  347.1931 ]
 [568.81335 347.65305]
 [501.47638 394.3781 ]
 [601.7341  382.81128]
 [470.68707 414.41473]
 [603.4693  400.4907 ]
 [528.66064 445.0612 ]
 [563.95856 444.89465]
 [520.5973  531.5599 ]
 [561.2191  529.67694]
 [529.24023 618.1388 ]
 [554.329   616.54895]]
e3cbffd_section_1_count_2.jpg
(2, 17, 2)
35134.8
[[645.4258  311.10294]
 [651.7254  303.59296]
 [644.57446 304.62228]
 [675.39233 305.42   ]
 [648.407   307.5708 ]
 [696.0746  344.44153]
 [660.5555  349.566  ]
 [675.68744 367.8999 ]
 [619.73535 377.72708]
 [631.05963 369.54852]
 [607.8273  375.7208 ]
 [710.6514  449.19037]
 [680.6335  448.94135]
 [697.09436 533.23303]
 [642.3748  517.81165]
 [719.8283  616.31134]
 [671.3116

In [71]:
import numpy as np

def normalize_keypoints_by_bbox_height(keypoints):
    """
    Normalize keypoints so that:
    - The top-left of the bounding box is (0, 0)
    - The bounding box height is 1
    """
    kp = np.array(keypoints)
    
    min_x = np.min(kp[:, 0])
    min_y = np.min(kp[:, 1])
    max_y = np.max(kp[:, 1])
    
    bbox_height = max_y - min_y
    if bbox_height < 1e-5:
        raise ValueError("Bounding box height too small for normalization.")
    
    # Translate to origin and scale
    kp_normalized = (kp - [min_x, min_y]) / bbox_height
    return kp_normalized, bbox_height


def compute_normalized_embeddings(leader_kp, follower_kp):
    leader_kp = np.array(leader_kp)
    follower_kp = np.array(follower_kp)

    # Normalize each person's keypoints by their own bbox height
    norm_leader = np.array([])
    norm_follower = np.array([])
    if np.sum(leader_kp) > 0:
        norm_leader, leader_height = normalize_keypoints_by_bbox_height(leader_kp)
    if np.sum(follower_kp) > 0:
        norm_follower, _ = normalize_keypoints_by_bbox_height(follower_kp)

    rel_vector = np.array([])
    if np.sum(leader_kp) > 0 and np.sum(follower_kp) > 0:
        # Get original centers (before normalization)
        center_leader = np.mean(leader_kp, axis=0)
        center_follower = np.mean(follower_kp, axis=0)
    
        # Compute relative position vector in leader's height units
        rel_vector = (center_follower - center_leader) / leader_height

    return norm_leader.flatten(), norm_follower.flatten(), rel_vector

In [73]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

for index in range(len(data[move_filename])):
    kp_leader = data[move_filename][index]['leader_keypoints']
    kp_follower = data[move_filename][index]['follower_keypoints']
    leader_embedding, follower_embedding, distance_embedding = compute_normalized_embeddings(kp_leader, kp_follower)
    data[move_filename][index]['leader_embedding'] = leader_embedding.tolist()
    data[move_filename][index]['follower_embedding'] = follower_embedding.tolist()
    data[move_filename][index]['distance_embedding'] = distance_embedding.tolist()

# Override the data with the new embeddings
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

In [75]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# Add musicality (TODO: This is manual)
musicality = [
    '-',
    '-',
    '-',
    '-',
    '~4',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '*',
    '-',
    '-',
    '-',
    '-'
]
for index in range(len(data[move_filename])):
    data[move_filename][index]['musicality'] = musicality[index]

# Override the data with musicality
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

In [47]:
import random
import numpy as np
import json
import os

moves_filepath = os.path.join('outputs', 'moves.json')
# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    moves_data = json.load(file)

# -------------------------
# 1. Build embeddings
# -------------------------
def build_embedding(move_item):
    """Return a 34+34+2 = 70 sized embedding"""

    l = move_item.get("leader_embedding", [])
    f = move_item.get("follower_embedding", [])
    d = move_item.get("distance_embedding", [])

    # pad or slice to fixed sizes
    l = (l + [0]*34)[:34]
    f = (f + [0]*34)[:34]
    d = (d + [0]*2)[:2]

    return np.array(l + f + d, dtype=float)

# -------------------------
# 2. Cosine Similarity
# -------------------------
def cosine_sim(a, b):
    if np.all(a == 0) or np.all(b == 0):
        return 0
    return np.dot(a, b) / (np.linalg.norm(a)*np.linalg.norm(b))

embedding_map = {x + 1: [] for x in range(8)}
for move_id in moves_data:
    current_move = moves_data[move_id]
    for index, move_item in enumerate(current_move):
        current_count = move_item['count']
        embedding_map[move_item['count']].append({
            'move_id': move_id,
            'index': index,
            'embedding': build_embedding(move_item),
            'is_candidate': len(current_move) - index >= 8 # I need at least an eight to include it in the video
        })

NUM_CONNECTIONS = 5
for count, items in embedding_map.items():
    for index, item in enumerate(items):
        current_embedding = item['embedding']
        embedding_candidates = [x['embedding'] for x in items]
        similarities = [(cosine_sim(current_embedding, embedding), idx) for idx, embedding in enumerate(embedding_candidates)]
        similarities = sorted(similarities, key=lambda x: x[0], reverse=True)
        move_id = item['move_id']
        move_index = item['index']
        moves_data[move_id][move_index]['connections'] =[]
        candidates_counter = 0
        for (similarity, candidate_index) in similarities:
            # The move has at least 8 steps and doesn't belong to the same video
            if items[candidate_index]['is_candidate'] and move_id != items[candidate_index]['move_id']:
                moves_data[move_id][move_index]['connections'].append({
                    'move_id': items[candidate_index]['move_id'],
                    'index': items[candidate_index]['index'],
                    'similarity': float(similarity)
                })
                candidates_counter += 1

            if candidates_counter >= NUM_CONNECTIONS:
                break

# Override the data with moves candidates per step
with open(moves_filepath, 'w') as fp:
    json.dump(moves_data, fp)

In [49]:
moves_data['ea90f11'][0]

{'section': 1,
 'count': 1,
 'time': 0.36,
 'leader_keypoints': [[381.48486328125, 349.853515625],
  [375.0570983886719, 328.75018310546875],
  [380.14251708984375, 334.749755859375],
  [322.619873046875, 336.7318115234375],
  [414.56378173828125, 344.21588134765625],
  [257.665771484375, 417.6979675292969],
  [459.3593444824219, 438.77435302734375],
  [183.65963745117188, 528.3660888671875],
  [513.2769775390625, 569.0078735351562],
  [250.53131103515625, 512.7069702148438],
  [429.0372314453125, 520.0631103515625],
  [293.06402587890625, 707.1112060546875],
  [412.6675720214844, 706.9732055664062],
  [233.6348876953125, 910.9539794921875],
  [496.6663818359375, 900.2425537109375],
  [158.32135009765625, 1114.271728515625],
  [582.3466796875, 1095.16259765625]],
 'follower_keypoints': [[381.48486328125, 349.853515625],
  [375.0570983886719, 328.75018310546875],
  [380.14251708984375, 334.749755859375],
  [322.619873046875, 336.7318115234375],
  [414.56378173828125, 344.21588134765625]